In [1]:
import pandas as pd
import numpy as np

import gpvisc

2024-07-03 14:02:21.678839: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Get data from sciglass

- we only take oxide melts
- we only take compositions that sum close to 1, between 99 and 100 %
- we drop compositions that contains other elements that those we seek
- we save the data in a CSV file

*If you don't want to run a SciGlass search, you can skip this cell and go to that below where we load the result of this database query directly.*

In [2]:
from glasspy.data import SciGlass
print(SciGlass.available_properties())

list_oxydes = ['SiO2','Al2O3','Na2O','K2O','MgO','CaO', 'FeO', 'TiO2', 'P2O5', 'Fe2O3', 'H2O', 'MnO']
# SciGlass lists interpolated values in the form of 
# temperatures at viscosities from log10 Pa s = 0 to log10 Pa s = 12
list_properties = ['T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12']

config_el = {
    "drop": ['S', 'C', 'Pt', 'Au', 'F', 'Cl', 'N', 'Br', 'I', 'Li', 
                                          'Be', 'B', 'Sc', 'V', 'Cr', 
                                          'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Rb', 'Sr', 'Y', 
                                          'Zr', 'Nb', 'Mo', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 
                                          'Te', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Sm', 'Eu', 'Gd', 'Tb', 
                                          'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Hg',
                                          'Tl', 'Pb', 'Bi', 'Th', 'U','Re','Ir'],}
config_prop = {"keep": list_properties} # we want only those properties
config_comp = {"keep": list_oxydes, # we want only those elements
              "acceptable_sum_deviation": 0.01} # the sum should be between 0.99 and 1.01

source = SciGlass(
    elements_cfg=config_el,
    properties_cfg=config_prop,
    compounds_cfg=config_comp,
)

df = source.data.reset_index()

###
# Generating an output dataframe
# in the same form that our dataset.
###

# correspondance array
corresp_visco = {"T0":0, 
                 "T1":1, 
                 "T2":2, 
                 "T3":3, 
                 "T4":4, 
                 "T5":5, 
                 "T6":6, 
                 "T7":7, 
                 "T8":8, 
                 "T9":9, 
                 "T10":10, 
                 "T11":11, 
                 "T12":12}

# the new dataframe
df2 = pd.DataFrame(columns=['sio2', 'tio2', 'al2o3', 'feo', 'fe2o3', 
                            'mno', 'na2o', 'k2o', 'mgo', 'cao', 'p2o5', 
                            'h2o', "T", "viscosity", "Author", "Year"])

# loop
for j in range(len(df)):
    
    # get the composition
    compo = df.compounds.loc[j,:]
    
    # get all properties
    prop = df.property.loc[j,:]
    
    # get author
    author = df.metadata.loc[j,"Author"]
    # get year
    year = df.metadata.loc[j,"Year"]
    
    # now loop over the desired property recorded visco
    for i in corresp_visco:
        
        # and only get T values that are not NaN
        if np.isnan(prop[i]) == False:   
            
            # construct new row
            new_row = [compo.SiO2, 
                       compo.TiO2, 
                       compo.Al2O3,
                       compo.FeO,
                       compo.Fe2O3,
                       compo.MnO, 
                       compo.Na2O, 
                       compo.K2O,
                       compo.MgO,
                       compo.CaO,
                       compo.P2O5,
                       compo.H2O, 
                       prop[i], # T of interest is in the correspondance dict
                       corresp_visco[i],
                      author,
                      year] # corresponding viscosity
            
            # append it to existing array
            df2.loc[len(df2)] = new_row
            
# save the dataframe
df2.to_csv("./data/FULL_SCIGLASS.csv")

['T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'Viscosity773K', 'Viscosity873K', 'Viscosity973K', 'Viscosity1073K', 'Viscosity1173K', 'Viscosity1273K', 'Viscosity1373K', 'Viscosity1473K', 'Viscosity1573K', 'Viscosity1673K', 'Viscosity1773K', 'Viscosity1873K', 'Viscosity2073K', 'Viscosity2273K', 'Viscosity2473K', 'Tg', 'Tmelt', 'Tliquidus', 'TLittletons', 'TAnnealing', 'Tstrain', 'Tsoft', 'TdilatometricSoftening', 'AbbeNum', 'RefractiveIndex', 'RefractiveIndexLow', 'RefractiveIndexHigh', 'MeanDispersion', 'Permittivity', 'TangentOfLossAngle', 'TresistivityIs1MOhm.m', 'Resistivity273K', 'Resistivity373K', 'Resistivity423K', 'Resistivity573K', 'Resistivity1073K', 'Resistivity1273K', 'Resistivity1473K', 'Resistivity1673K', 'YoungModulus', 'ShearModulus', 'Microhardness', 'PoissonRatio', 'Density293K', 'Density1073K', 'Density1273K', 'Density1473K', 'Density1673K', 'ThermalConductivity', 'ThermalShockRes', 'CTEbelowTg', 'CTE328K', 'CTE373K', 'CTE433K', 'CT

## Reading the dataframe

Now we don't need SciGlass anymore, we just load the dataframe.

In [11]:
data_glasspy = pd.read_csv("./additional_data/FULL_SCIGLASS.csv")
data_glasspy['Name'] = gpvisc.generate_token(data_glasspy) # We add the unique names
data_glasspy=data_glasspy.drop(['Unnamed: 0','Author', 'Year'], axis=1)
data_glasspy["Sciglass"]=True
data_glasspy["Name_publi"]="Sciglass"
data_glasspy["ref"]="Sciglass"
# print columns
data_glasspy.columns

Index(['sio2', 'tio2', 'al2o3', 'feo', 'fe2o3', 'mno', 'na2o', 'k2o', 'mgo',
       'cao', 'p2o5', 'h2o', 'T', 'viscosity', 'Name', 'Sciglass',
       'Name_publi', 'ref'],
      dtype='object')

we load our data

In [12]:
dataset_lp = pd.read_excel("./additional_data/database.xlsx", sheet_name="lp")
dataset_lp['Name'] = gpvisc.generate_token(dataset_lp) # We add the unique names
dataset_lp["Sciglass"]=False
dataset_lp = dataset_lp.loc[:,data_glasspy.columns] # we get only the same columns as in data_glasspy

Now we will create a new dataframe in which we keep everything from dataset_lp and only add the compositions from data_glasspy that do not appear in dataset_lp (at a resolution of 0.1 mol%)

In [13]:
# Merge df2 with df1 with an indicator and perform a left join
merge_df = pd.merge(data_glasspy, dataset_lp, on=['sio2','tio2','al2o3','feo','fe2o3','mno','na2o','k2o','mgo','cao','p2o5','h2o'], 
                    how='left', indicator=True)

# Filter out rows in df2 that are found in df1
unique_df2 = merge_df[merge_df['_merge'] == 'left_only'].drop(columns=['_merge'])

# we clean unique_df2 and drop the details of the merge
unique_df2['T'] = unique_df2['T_x']
unique_df2['Name'] = unique_df2['Name_x']
unique_df2['viscosity'] = unique_df2['viscosity_x']
unique_df2 = unique_df2.drop(['T_x','T_y','viscosity_x','viscosity_y','Name_x','Name_y','Sciglass_x','Sciglass_y'],axis=1).copy()

# Concatenate unique rows from df2 with df1
final_df = pd.concat([dataset_lp, unique_df2]).reset_index()

# add pressure, everything at 0 GPa (1 bar)
final_df["P"] = 0.0

# print details
print("Old length {}, added data {}, new length {}".format(len(dataset_lp),len(unique_df2), len(final_df)))

# save dataframe
final_df.drop("index",axis=1).to_csv("./additional_data/dataset+sciglass_lp.csv")

Old length 15440, added data 12231, new length 27671


In [14]:
unique_df2.loc[:,"Name"].drop_duplicates()

80         S75.3T0.0A0.4F0.0M0.0N14.3K0.0Mg0.2C9.8P0.0H0.0
85        S70.8T0.0A1.2F0.8M0.1N15.7K0.5Mg0.0C10.9P0.0H0.0
89         S70.0T0.0A0.0F0.0M0.0N20.4K0.0Mg0.0C9.6P0.0H0.0
91        S60.0T0.0A16.0F0.0M0.0N0.0K16.0Mg0.0C8.0P0.0H0.0
94        S60.0T0.0A16.0F0.0M0.0N3.2K12.8Mg0.0C8.0P0.0H0.0
                               ...                        
55924      S35.7T1.1A9.0F0.0M3.0N0.0K0.0Mg9.1C42.1P0.0H0.0
55925      S36.2T1.9A9.0F0.0M1.3N0.0K0.0Mg9.1C42.6P0.0H0.0
55926    S31.7T2.9A10.8F0.6M0.3N0.0K0.0Mg13.8C40.0P0.0H0.0
55927    S31.0T2.8A12.7F0.6M0.3N0.0K0.0Mg13.5C39.1P0.0H0.0
55928     S67.2T0.0A2.6F0.0M0.0N14.3K0.6Mg11.3C4.0P0.0H0.0
Name: Name, Length: 3591, dtype: object

In [15]:
len(final_df.loc[:,gpvisc.list_oxides()])

27671

In [16]:
final_df.isnull().all()

index           False
sio2            False
tio2            False
al2o3           False
feo             False
fe2o3           False
mno             False
na2o            False
k2o             False
mgo             False
cao             False
p2o5            False
h2o             False
T               False
viscosity       False
Name            False
Sciglass        False
Name_publi      False
ref             False
Name_publi_x    False
ref_x           False
Name_publi_y     True
ref_y            True
P               False
dtype: bool

## Loading and saving the HP data in CSV

It is faster to load CSV files. So we resave our data in CSV for futur use, particularly in the loop that select random seed (see second data preparation notebook)

In [17]:
dataset_hp = pd.read_excel("./additional_data/database.xlsx", sheet_name="hp")
dataset_hp["Sciglass"] = False
dataset_hp.to_csv("./additional_data/dataset_hp.csv")

In [18]:
dataset_lp.loc[:,gpvisc.list_oxides()].max()

sio2     100.00
tio2      37.26
al2o3    100.00
feo       71.50
fe2o3     82.89
mno       31.53
na2o      54.15
k2o       50.00
mgo       65.10
cao       74.70
p2o5       6.67
h2o      100.00
dtype: float64

In [19]:
unique_df2.loc[:,gpvisc.list_oxides()].min()

sio2     0.0
tio2     0.0
al2o3    0.0
feo      0.0
fe2o3    0.0
mno      0.0
na2o     0.0
k2o      0.0
mgo      0.0
cao      0.0
p2o5     0.0
h2o      0.0
dtype: float64